In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import L2

from dataloader_v1 import SpectDataset
import numpy as np
from sklearn.metrics import classification_report
import itertools

In [4]:
def SBCNN_Model(input_shape, num_labels):
    model = Sequential()
    model.add(Input(input_shape))
    model.add(Conv2D(24, 5, padding='same'))
    model.add(MaxPool2D(strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, 5, padding='same'))
    model.add(MaxPool2D(strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, 5, padding='valid'))
    model.add(Activation('relu'))

    model.add(Flatten())

    model.add(Dropout(0.5))
    model.add(Dense(64, kernel_regularizer=L2(0.001)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.5))
    model.add(Dense(num_labels, kernel_regularizer=L2(0.001)))
    model.add(Activation('softmax'))

    return model

In [ ]:
def run_model(optimizer, n_iter):
    DATA_PATH = 'D:\\SP Cup 2022\\New folder\\spcup_2022_training_part1'
    Labeled_dir = 'D:\\SP Cup 2022\\New folder\\spcup_2022_training_part1\\labels.csv'

    dataloader = SpectDataset(add_noise=True)
    train_ds, val_ds = dataloader.call(DATA_PATH, Labeled_dir)

    for spectrogram, _ in train_ds.take(1):
        input_shape = spectrogram.shape
    model = SBCNN_Model(input_shape[1:], 6)

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )
    EPOCHS = n_iter
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
    # callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
    )
    res_sparse = model.predict(val_ds)
    res_np = np.argmax(res_sparse, axis=1)
    pred_labels = list(res_np)
    true_labels = []
    for _, label in val_ds.take(1000):
        true_labels.extend(list(label.numpy()))
        
    print("SBCNN with optimizer {} and {} iterations".format(optimizer, n_iter))
    class_names = [f'class_{i}' for i in range(6)]
    print(classification_report(true_labels, pred_labels, target_names=class_names))

In [6]:
sgd = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
optimizer = [sgd, 'adam']
n_iter = [10, 15]

params = list(itertools.product(optimizer, n_iter))

for p in params:
    run_model(p[0], p[1])

C:\Users\smcha\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
